# Milestone 1 (Group: Randomizers)

In [1]:
import json

with open('./ir-anthology-07-11-2021-ss23.jsonl', 'r') as json_file:
    json_list = list(json_file)

lis = []
for json_str in json_list:
    result = json.loads(json_str)
    id = result["id"]
    lis.append({"doc_id" : id, "text" : f"{result}"})
    #  das text-feld is momentan der gesamte eintrag an sich, das sollten wir noch ändern!

with open("ir-anthology-processed.jsonl", 'w') as f:
    for item in lis:
        f.write(json.dumps(item) + "\n")

# JSONL processing
This first cell reads the given .jsonl file and processes it into a new one of the desired form.